[Reference](https://medium.com/@lironbenyeda/fastapi-sqlalchemy-and-parallel-queries-walk-into-a-bar-86dfe40aa878)

In [2]:
async def get_db_session():
    db_session = DBSession()
    try:
        yield db_session
    finally:
        db_session.close()

@app.get("/total_books")
async def get_total_books(db_session: AsyncSession = Depends(get_db_session)):
    return await crud.get_book_table_total_count(db_session)

In [3]:
async def get_book_table_total_count(db_session: AsyncSession):
    result = await db_session.execute(select(count()).select_from(Book))
    return result.scalar() or 0

In [4]:
@app.get("/multiple_counts")
async def get_multiple_counts(db_session: AsyncSession = Depends(get_db_session)):
    async with TaskGroup() as tg:
        task1 = tg.create_task(crud.get_book_table_total_count(db_session))
        task2 = tg.create_task(crud.get_book_table_total_count(db_session))
        task3 = tg.create_task(crud.get_book_table_total_count(db_session))
    return task1.result(), task2.result(), task3.result()

In [5]:
from asyncio import current_task

from sqlalchemy.ext.asyncio import async_scoped_session, async_sessionmaker

async_session_factory = async_sessionmaker(
    some_async_engine, expire_on_commit=False
)
AsyncScopedSession = async_scoped_session(
    async_session_factory, scopefunc=current_task
)
some_async_session = AsyncScopedSession()

In [6]:
from asyncio import current_task, TaskGroup
from typing import AsyncGenerator, Iterable

from sqlalchemy.ext.asyncio import AsyncSession
from sqlalchemy.ext.asyncio import async_scoped_session

from db.db_connector import SqlAlchemyConnector

class DBManager:
    def __init__(self):
        connector = SqlAlchemyConnector()
        self.scoped_session_factory = \
            async_scoped_session(connector.session_factory,
                                 scopefunc=_get_current_task_id)

    def get_session(self) -> AsyncSession:
        return self.scoped_session_factory()


# New dependency
async def get_db_manager() -> AsyncGenerator[DBManager, None]:
    db_manager = DBManager()
    try:
        yield db_manager
    finally:
        sessions = db_manager.scoped_session_factory.registry.registry.values()
        await _close_sessions(sessions)


async def _close_sessions(db_sessions: Iterable[AsyncSession]):
    async with TaskGroup() as task_group:
        for db_session in db_sessions:
            task_group.create_task(db_session.close())


def _get_current_task_id() -> int:
    return id(current_task())